##Question 2 Final Exam:

Using Agent Pro, build a real, functioning AI Agent leveraging a Multi-Agent Architecture to solve a real world problem.

Answer:
"Tool-Based Multi-Agent Q&A Assistant"

A working chatbot system with:

🔹 CalculatorBot – for math expressions

🔹 ARES TOOL – for Internet Search

🔹 DUCK DUCK GO  – for For Youtube Video Links

🔹 WikiBot – for general knowledge searching on Wikipedia

🔹 MistralBot – an LLM answering complex questions

🔹 Main Code

In [9]:
# Install the Hugging Face 'transformers' library for working with pre-trained language models
# Install 'accelerate' to simplify model training and hardware utilization (e.g., GPU/TPU)
# Install 'wikipedia' to enable searching and fetching content from Wikipedia programmatically
!pip install transformers accelerate wikipedia

In [10]:
# Install the 'requests' library, which is used to send HTTP/1.1 requests easily in Python
# Useful for making API calls, downloading web content, and interacting with web services
!pip install requests

Youtube Class

In [11]:
# Install the 'duckduckgo-search' library, which provides a simple interface
# to perform searches using the DuckDuckGo search engine from Python
!pip install duckduckgo-search

In [12]:
# Import DDGS (DuckDuckGo Search) class to perform searches including videos
from duckduckgo_search import DDGS

# Import utilities to parse URLs and query strings
from urllib.parse import urlparse, parse_qs

# Define a custom class to perform YouTube searches using DuckDuckGo
class YouTubeSearchTool:

    # Constructor method: initialize the DDGS search instance
    def __init__(self):
        self.ddgs = DDGS()

    # Helper method to extract a YouTube video ID from a given URL
    def extract_video_id(self, url):
        parsed_url = urlparse(url)  # Parse the input URL
        # Handle standard YouTube watch links
        if parsed_url.hostname in ['www.youtube.com', 'youtube.com']:
            if parsed_url.path == '/watch':
                # Extract video ID from the query string parameter 'v'
                return parse_qs(parsed_url.query)['v'][0]
            elif parsed_url.path.startswith('/shorts/'):
                # Extract video ID from YouTube Shorts URLs
                return parsed_url.path.split('/')[2]
        # Handle shortened YouTube links (youtu.be)
        elif parsed_url.hostname == 'youtu.be':
            return parsed_url.path[1:]
        # If the URL is not recognized as a YouTube link
        return None

    # Method to perform a YouTube video search using DuckDuckGo
    def search_videos(self, query, max_results=3):
        try:
            # Perform the video search with custom filters
            results = self.ddgs.videos(
                keywords=query,       # Search query
                region="wt-wt",       # Region (wt-wt is worldwide)
                safesearch="off",     # Disable safe search filtering
                timelimit="w",        # Limit to videos from the past week
                resolution="high",    # Prefer high-resolution videos
                duration="medium",    # Prefer medium-duration videos
                max_results=max_results * 2  # Fetch more than needed for filtering
            )
            # Sort results by view count in descending order
            results = sorted(results, key=lambda x: -(x.get('statistics', {}).get('viewCount', 0)))

            videos = []  # List to store filtered video info
            for result in results[:max_results]:  # Limit to the top N results
                video_url = result.get('content')  # Extract video URL
                video_id = self.extract_video_id(video_url)  # Extract video ID
                if video_id:
                    # Build a structured video info dictionary
                    videos.append({
                        'title': result['title'],  # Video title
                        'video_id': video_id,      # YouTube video ID
                        'description': result.get('description', ''),  # Video description
                        'link': video_url,         # Full video URL
                        'view_count': result.get('statistics', {}).get('viewCount', 'N/A')  # View count
                    })
            # Return the list of videos, or a message if none were found
            return videos or "No relevant YouTube videos found."

        # Handle and return any errors that occur during search
        except Exception as e:
            return f"Error during YouTube search: {str(e)}"


Loading Mistralai Model as Base Agent

In [14]:
# Import the Hugging Face Transformers tokenizer and model for causal language modeling
from transformers import AutoTokenizer, AutoModelForCausalLM

# Import PyTorch for tensor operations and device management
import torch

# Import regular expressions module for parsing/calculations
import re

# Import Wikipedia library for fetching summaries from Wikipedia
import wikipedia

# Import OS library (currently unused in the code but may be useful later)
import os


# Define a class to interact with the Mistral-7B-Instruct language model
class MistralBot:
    # Constructor: load the tokenizer and model from Hugging Face
    def __init__(self, model_name="mistralai/Mistral-7B-Instruct-v0.2"):
        print("Loading Mistral-7B-Instruct model...")
        # Load tokenizer for the specified model
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        # Load the language model with float16 precision and automatic device placement (e.g., GPU/CPU)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        print("Model loaded.")

    # Generate text from the model using the given prompt
    def generate(self, prompt, max_tokens=150):
        # Tokenize the prompt and move it to the same device as the model
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
        # Generate output using the model with sampling
        outputs = self.model.generate(**inputs, max_new_tokens=max_tokens, do_sample=True)
        # Decode the generated tokens to text
        text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        # Extract only the portion after "Final Answer:" (if exists)
        return text.split("Final Answer:")[-1].strip()


# Define an agent that uses tools and an LLM to answer questions
class ToolBasedAgent:
    # Constructor: accept an instance of MistralBot or similar model
    def __init__(self, llm_agent):
        self.llm_model = llm_agent  # Assign the language model

    # Calculator tool: extract and evaluate basic math expressions from a string
    def calculator(self, question):
        # Use regex to find a simple arithmetic expression (e.g., "12 + 5")
        match = re.findall(r"[-+]?\d+\.?\d*\s*[\+\-\*/]\s*[-+]?\d+\.?\d*", question)
        if match:
            try:
                # Evaluate the first matched expression safely
                return str(eval(match[0]))
            except:
                return "Calculation error."
        return None

    # Wikipedia tool: fetch a short summary for a given topic
    def wiki(self, question):
        try:
            # Return the first two sentences from Wikipedia summary
            return wikipedia.summary(question, sentences=2)
        except:
            return None  # Return None if the topic isn't found or any error occurs

    # Run the language model with a prompt
    def run_llm(self, prompt):
        return self.llm_model.generate(prompt)

    # Web search tool using Ares API
    def web_search(self, query):
        try:
            url = "https://api.aresapi.com/search"  # Ares API endpoint
            headers = {
                "X-API-Key": self.ares_api_key  # API key for authentication
            }
            params = {
                "q": query,       # Query string
                "num": 3          # Limit results to top 3
            }
            # Make GET request to Ares API
            response = requests.get(url, headers=headers, params=params)
            # Extract result list from JSON response
            results = response.json().get("results", [])
            if not results:
                return "No web results found."
            # Format results as bullet points with title and URL
            return "\n".join([f"- {r['title']}: {r['url']}" for r in results])
        except Exception as e:
            # Handle any API or network errors
            return f"Ares error: {str(e)}"

    # YouTube search tool
    def youtube_search(self, query):
        return self.youtube_tool.search_videos(query)


React Loop

In [15]:
# Define the ReAct reasoning loop function
# agent: an instance of ToolBasedAgent with tools like calculator, wiki, and LLM
# user_question: the original question asked by the user
# max_steps: how many reasoning steps to allow before forcing a final answer
def react_loop(agent, user_question, max_steps=5):

    # Description of tools available to the agent, included in the system prompt
    tools_description = """
Calculator: useful for solving math problems and equations.
Wiki: useful for retrieving short summaries from Wikipedia.
"""

    # The list of valid tool/action names, including the final answer
    tool_names = "Calculator, Wiki, Answer"

    # System prompt that defines the ReAct-style instruction format for the agent
    system_prompt = f"""
Answer the following questions as best you can. You have access to the following tools:

{tools_description}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
"""

    # Initialize the conversation history with the user question
    history = f"Question: {user_question}\n"

    # Begin the reasoning loop for up to `max_steps` iterations
    for _ in range(max_steps):
        # Construct the prompt for the LLM by appending current history to the system prompt
        prompt = system_prompt + history
        # Run the LLM on the prompt to get the next reasoning step
        response = agent.run_llm(prompt)

        # Convert response to lowercase for easier matching
        lower_resp = response.lower()

        # Check which tool was chosen by the agent
        if "action: calculator" in lower_resp:
            action = "Calculator"
            action_input = user_question  # In this simplified version, we reuse the original question
            observation = agent.calculator(action_input)  # Call the calculator tool

        elif "action: wiki" in lower_resp:
            action = "Wiki"
            action_input = user_question
            observation = agent.wiki(action_input)  # Call the Wikipedia tool

        elif "final answer" in lower_resp:
            # If the model directly gives the final answer, return it
            return response.strip()

        else:
            # If the model doesn’t follow the expected format, stop and return what it gave
            return f"Final Answer: {response.strip()}"

        # Update the reasoning history with the agent's response and tool observation
        history += f"{response.strip()}\nObservation: {observation}\n"

    # If max_steps are exhausted and no final answer was given,
    # explicitly ask the model to produce the final answer
    final_prompt = system_prompt + history + "Thought: I now know the final answer\nFinal Answer:"
    final_response = agent.run_llm(final_prompt)

    return final_response.strip()


Run the Multi Agent System

In [8]:
# Instantiate the MistralBot language model (loads the Mistral-7B-Instruct model)
mistral = MistralBot()

# Create an instance of ToolBasedAgent and pass the Mistral model to it
# This agent can now use LLM responses along with tools like calculator and Wikipedia
agent = ToolBasedAgent(mistral)

# Define the user's question to be answered using the reasoning loop
user_question = "What is Newton's second law of motion?"

# Run the ReAct-style reasoning loop with the agent to get a final response
final_response = react_loop(agent, user_question)

# Print the final answer returned by the reasoning loop
print("FINAL ANSWER:\n", final_response)


Loading Mistral-7B-Instruct model...


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Model loaded.
FINAL ANSWER:
 Final Answer: Newton's second law of motion states that the net force acting on an object is equal to the mass of that object multiplied by its acceleration (F = ma).
